In [1]:
pwd

'/Users/benhayum/Desktop/Classes/Junior/Fall/CS 540/hw4/HW4'

In [2]:
## Part 1 -- load_data

In [3]:
import csv

In [4]:
def load_data(filepath):
    list_dictionaries = list()
    with open(filepath) as csv:
        for line in csv:
            curr_dictionary = dict()
            row_entries = line.split(",")
            if row_entries[0] != "#":
                curr_dictionary["HP"] = row_entries[5]
                curr_dictionary["Attack"] = row_entries[6]
                curr_dictionary["Defense"] = row_entries[7]
                curr_dictionary["Sp. Atk"] = row_entries[8]
                curr_dictionary["Sp. Def"] = row_entries[9]
                curr_dictionary["Speed"] = row_entries[10]
                list_dictionaries.append(curr_dictionary)

    return list_dictionaries

In [5]:
all_dictionaries = load_data("Pokemon.csv")

In [36]:
len(all_dictionaries)

800

In [6]:
## Part 2 -- calc_features

In [7]:
import numpy as np

In [8]:
row1 = load_data("Pokemon.csv")[0]
row1
# Number Pokemon gets repeated sometimes in the data for e.g. Mega versions of same Pokemon

{'HP': '45',
 'Attack': '49',
 'Defense': '49',
 'Sp. Atk': '65',
 'Sp. Def': '65',
 'Speed': '45'}

In [9]:
feature_representation = np.zeros(6, dtype=np.int64)
feature_representation[0] = row1["HP"]
feature_representation[1] = row1["Attack"]
feature_representation[2] = row1["Defense"]
feature_representation[3] = row1["Sp. Atk"]
feature_representation[4] = row1["Sp. Def"]
feature_representation[5] = row1["Speed"]
feature_representation

array([45, 49, 49, 65, 65, 45])

In [10]:
feature_representation.dtype

dtype('int64')

In [11]:
# Putting it all into a function:
def calc_features(row):
    feature_representation = np.zeros(6, dtype=np.int64)
    feature_representation[0] = row["HP"]
    feature_representation[1] = row["Attack"]
    feature_representation[2] = row["Defense"]
    feature_representation[3] = row["Sp. Atk"]
    feature_representation[4] = row["Sp. Def"]
    feature_representation[5] = row["Speed"]
    return feature_representation

In [12]:
calc_features(row1)

array([45, 49, 49, 65, 65, 45])

In [13]:
## Part 3 -- hac

In [14]:
# Hierarchical Clustering
# MAINTAIN A DISTANCE MATRIX THROUGHOUT THE PROCESS
#   Point index to every other point -- like functional connectivity

In [15]:
# Building a list of all of the points
all_points = list()
for i in range(len(all_dictionaries)):
    curr_point = calc_features(all_dictionaries[i])
    all_points.append(curr_point)
all_points

[array([45, 49, 49, 65, 65, 45]),
 array([60, 62, 63, 80, 80, 60]),
 array([ 80,  82,  83, 100, 100,  80]),
 array([ 80, 100, 123, 122, 120,  80]),
 array([39, 52, 43, 60, 50, 65]),
 array([58, 64, 58, 80, 65, 80]),
 array([ 78,  84,  78, 109,  85, 100]),
 array([ 78, 130, 111, 130,  85, 100]),
 array([ 78, 104,  78, 159, 115, 100]),
 array([44, 48, 65, 50, 64, 43]),
 array([59, 63, 80, 65, 80, 58]),
 array([ 79,  83, 100,  85, 105,  78]),
 array([ 79, 103, 120, 135, 115,  78]),
 array([45, 30, 35, 20, 20, 45]),
 array([50, 20, 55, 25, 25, 30]),
 array([60, 45, 50, 90, 80, 70]),
 array([40, 35, 30, 20, 20, 50]),
 array([45, 25, 50, 25, 25, 35]),
 array([65, 90, 40, 45, 80, 75]),
 array([ 65, 150,  40,  15,  80, 145]),
 array([40, 45, 40, 35, 35, 56]),
 array([63, 60, 55, 50, 50, 71]),
 array([ 83,  80,  75,  70,  70, 101]),
 array([ 83,  80,  80, 135,  80, 121]),
 array([30, 56, 35, 25, 35, 72]),
 array([55, 81, 60, 50, 70, 97]),
 array([40, 60, 30, 31, 31, 70]),
 array([ 65,  90,  65,

In [16]:
distance_matrix = np.zeros( (len(all_points),len(all_points))  )

In [17]:
for i in range(len(all_points)):
    i_point = all_points[i]
    for j in range(len(all_points)):
        j_point = all_points[j]

        distance_matrix[i, j] = np.linalg.norm(i_point - j_point)
    distance_matrix[i, i] = np.inf
distance_matrix

array([[         inf,  35.56683849,  84.52810184, ..., 130.92746083,
        173.61451552, 124.34629066],
       [ 35.56683849,          inf,  48.98979486, ..., 101.05938848,
        144.95861478,  93.02150289],
       [ 84.52810184,  48.98979486,          inf, ...,  69.37578828,
        111.41364369,  57.03507693],
       ...,
       [130.92746083, 101.05938848,  69.37578828, ...,          inf,
         54.77225575,  74.83314774],
       [173.61451552, 144.95861478, 111.41364369, ...,  54.77225575,
                 inf,  96.95359715],
       [124.34629066,  93.02150289,  57.03507693, ...,  74.83314774,
         96.95359715,          inf]])

In [18]:
# 1st column is cluster1 points
# 2nd column is cluster2 points
# 3rd column is complete linkage distance
# 4th column is number of pokemon
# 5th column is curr_distance_matrix indices of all cluster1 points
# 6th column is curr_distance_matrix indices of all cluster2 points
Z = np.zeros((len(all_points), 6))

In [23]:
class cluster:
    def __init__(self, indices, points):
        self.indices = indices
        self.points = points

In [24]:
cluster_list = list()
for n in range(len(all_points)):
    curr_cluster = cluster(n, all_points[n])
    cluster_list.append(curr_cluster)

In [25]:
# Keep a dictionary of clusters to index of distance array points
cluster2index = dict()
for i_cluster in range(len(cluster_list)):
    cluster2index[ cluster_list[i_cluster] ] = i_cluster
cluster2index

{<__main__.cluster at 0x7fc79c5c2d90>: 0,
 <__main__.cluster at 0x7fc79c5c2a60>: 1,
 <__main__.cluster at 0x7fc79c5c23a0>: 2,
 <__main__.cluster at 0x7fc79c5c2fd0>: 3,
 <__main__.cluster at 0x7fc79c5c2970>: 4,
 <__main__.cluster at 0x7fc79c5c2be0>: 5,
 <__main__.cluster at 0x7fc79c5c2610>: 6,
 <__main__.cluster at 0x7fc79c5c2a90>: 7,
 <__main__.cluster at 0x7fc79c5c2460>: 8,
 <__main__.cluster at 0x7fc79c5c2e50>: 9,
 <__main__.cluster at 0x7fc79c5c29d0>: 10,
 <__main__.cluster at 0x7fc79c5c2100>: 11,
 <__main__.cluster at 0x7fc79c5c24c0>: 12,
 <__main__.cluster at 0x7fc79c5c28b0>: 13,
 <__main__.cluster at 0x7fc79c5c2130>: 14,
 <__main__.cluster at 0x7fc79c5c2250>: 15,
 <__main__.cluster at 0x7fc79c5c2640>: 16,
 <__main__.cluster at 0x7fc79c5c20d0>: 17,
 <__main__.cluster at 0x7fc79a165f70>: 18,
 <__main__.cluster at 0x7fc79a1652b0>: 19,
 <__main__.cluster at 0x7fc79a165940>: 20,
 <__main__.cluster at 0x7fc79a1659a0>: 21,
 <__main__.cluster at 0x7fc79a1650a0>: 22,
 <__main__.cluster at

In [27]:
cluster_list[1].points

array([60, 62, 63, 80, 80, 60])

In [28]:
curr_min = np.inf
row_index = -1
column_index = -1
for i in range(len(distance_matrix)):
    for j in range(len(distance_matrix)):
        if distance_matrix[i, j] < curr_min:
            curr_min = distance_matrix[i, j]
            row_index = i
            column_index = j
(row_index, column_index)

(4, 169)

In [30]:
x = type(cluster_list[row_index].points)
x == np.ndarray

True

In [ ]:
iteration = 1

In [ ]:
# ONLY 1 BELOW BUT IT'LL BE THE NUMBER OF ITERATIONS of clusters created
    # number of i's is length of features - 1
curr_index = len(all_points) + iteration
prev_cluster1 = cluster_list[row_index].points
prev_cluster2 = cluster_list[column_index].points
curr_points = list()
if type(prev_cluster1) == np.ndarray:
    curr_points.append(prev_cluster1)
else:
    for k in range(len(prev_cluster1)):
        curr_points.append(prev_cluster1[k])
if type(prev_cluster2) == np.ndarray:
    curr_points.append(prev_cluster2)
else:
    for k in range(len(prev_cluster2)):
        curr_points.append(prev_cluster2[k])
new_cluster = cluster(curr_ID, curr_points)
cluster_list.append(new_cluster)

cluster_list.pop(row_index)
cluster_list.pop(column_index)

In [ ]:
cluster_list[-1].point

In [34]:
x = list([1, 2, 3, 4])
print(x.pop(1))
x

[1, 3, 4]

In [35]:
print(x.pop(2))

IndexError: pop index out of range

In [ ]:
def update_distance_matrix(distance_matrix, cluster_list, new_cluster):
    # Keep it the same size and fill all in the new cluster with the new complete linkage

    for i in range(len(cluster_list)):
        i_curr_cluster = cluster_list[i]
        if i_curr_cluster in cluster2index.keys():
            i_curr_index = cluster2index[i_curr_cluster]

            for j in range(len(cluster_list)):
                j_curr_cluster = cluster_list[j]
                if j_curr_cluster in cluster2index.keys():
                    j_curr_index = cluster2index[j_curr_cluster]




    # New cluster gets added onto the last index with all of its complete linkages in the FC matrix



In [ ]:
# Keep distance matrix the same -- have an array with the indices of the points merged
# -- call argmax on these points for each merge


In [ ]:
# Distances of clusters = min(row_indices, column_indices)??
# Find index of smallest distance
# Return result matrix Z
# Create a whole new distance matrix after every merge?
    #z 0,1 are the actual points
    # when multiple points in one cluster, have an array of points which are themselves arrays
# Create a cluster class which can store both the data and cluster ID in one object?? Not necessarily needed.